# Read-out Error


In [1]:
# import packages
import numpy as np

import qiskit_aer.noise as noise
from qiskit_aer.noise.errors import coherent_unitary_error, amplitude_damping_error, ReadoutError
from qiskit.circuit.library import RXGate

from qiskit_aer.primitives import SamplerV2 as Sampler
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp

from readout_error import convert_probabilities

In [ ]:
def convert_probabilities(X):
    """
    args:
        X: np.darray (Nx2) probability of false result from most to least significant bit
    returns:
        output: np.darray (2^N x 2^N) matrix of readout errors
    """
    # cast to np array
    try:
        X = np.array(X)
    except Exception as exception:
        print("Must pass in valid datatype: array of numbers.")
        print(exception)
    
    # check array dimensions
    D = X.shape[1]
    if D != 2:
        raise ValueError("Input probabilities must be in form [[probability of error when prepared 0, probability of error when prepared 1], ...]")

    # number data points
    N = X.shape[0]

    output = np.ones((1, 1))

    SIGN = np.array([[-1, 1], [1, -1]])

    # goal shape: 2^N x 2^N
    for i in range(N):
        # obtain confusion matrix
        p_i = X[i].reshape((2, 1)) # [[FN], [FP]]
        p_i = p_i * SIGN # [[-FN, FN], [FP, -FP]]
        p_i = p_i + np.eye(2) # [[TN, FN], [FP, TP]]

        # reshape matrices
        p_i_flattened = p_i.reshape((1, p_i.size))

        # recursively calculate dimensions
        output_new = np.ones((2 * output.shape[0], 2 * output.shape[0])) 

        # i
        for j in range(output.shape[0]):
            row = output[j].reshape((output[j].size, 1))

            prod = row * p_i_flattened # shape output[j].size x 4
            output_new[2 * j] = prod[:, 0:2]
            output_new[2 * j + 1] = prod[:, 2:4]

        output = output_new

    return output